In [1]:
# !pip install scrapy
# !pip install OleFileIO_PL
# !pip install urllib

In [2]:
import scrapy
from urllib.parse import urlencode
import ast
from datetime import datetime
import OleFileIO_PL
import pandas as pd

In [3]:
import os
import pickle
from datetime import datetime

class FileAdmin():
    
    files_dir = '../files/'
    
    @classmethod
    def apaga(cls, spider, tipo):
        for file in os.listdir(cls.files_dir):
            x = file.split('.')
            if x[0] == spider and x[1] == tipo:
                os.remove(cls.files_dir + file)

    def _geranome(spider, name):
        # Gera e retorna o nome do arquivo
        marca_data = datetime.today().strftime('%Y-%m-%d')
        return f'{spider}.{name}.{marca_data}.p'
    
    @classmethod
    def grava(cls, dados, spider, name):
        filename = cls._geranome(spider, name)
        with open(cls.files_dir + filename, 'wb') as handle:
            pickle.dump(dados, handle)


In [4]:
class CepeaSpider(scrapy.Spider):
    
    name = "cepea"
    
    def debug(self):
        print(self.name)
        
    def __new__(cls, codinome, cod, dt_ini = '01/01/1990', dt_fim = datetime.today().strftime('%d/%m/%Y')):
        instance = object.__new__(cls)
        instance.codinome = codinome
        # start_urls
        instance.params = {
        'tabela_id' : cod,
        'data_inicial' : dt_ini,
        'periodicidade' : 1,
        'data_final' : dt_fim,
        }
        instance.start_urls = [
            'https://www.cepea.esalq.usp.br/br/consultas-ao-banco-de-dados-do-site.aspx?' + urlencode(instance.params),
        ]
        return instance
    
    def parse(self, response):
        final_url = ast.literal_eval(response.text)
        arquivo = final_url['arquivo'].replace("\\", "")
        yield scrapy.Request(
            url = arquivo,
            callback = self.save
        )

    def save(self, response):
        # Prepara resposta
        ole = OleFileIO_PL.OleFileIO(response.body)
        df = pd.read_excel(ole.openstream('Workbook'), skiprows=3)
        df = df.set_index("Data")
        
        # Apaga arquivo anterior
        FileAdmin.apaga(self.name, self.codinome)
        
        # Escreve novo arquivo
        FileAdmin.grava(
            df, 
            self.name,    # spider name
            self.codinome # codigo nome
        )

In [5]:
import yaml
from scrapy.crawler import CrawlerProcess
import time

# Load Settings
name = CepeaSpider.name
with open(f"{name}.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

# Run spider
for k, v in config.items():
    time.sleep(10)
    process = CrawlerProcess({
        'USER_AGENT': 'Mozilla/5.0 (compatible; MSIE 7.0; Windows NT 5.0)'
    })
    process.crawl(CepeaSpider, codinome=k, cod=v)

process.start()

2020-12-19 08:18:18 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-19 08:18:18 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.7, Platform Windows-10-10.0.19041-SP0
2020-12-19 08:18:18 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-12-19 08:18:18 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/5.0 (compatible; MSIE 7.0; Windows NT 5.0)'}
2020-12-19 08:18:18 [scrapy.extensions.telnet] INFO: Telnet Password: bb24ddf7fd239372
2020-12-19 08:18:18 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-12-19 08:18:18 [scrapy.middleware] INFO: Enabled downloader middlewares:
[

2020-12-19 08:18:48 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']
2020-12-19 08:18:48 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2020-12-19 08:18:48 [scrapy.core.engine] INFO: Spider opened
2020-12-19 08:18:48 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2020-12-19 08:18:48 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6026
2020-12-19 08:18:58 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-19 08:18:58 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (A

2020-12-19 08:19:40 [scrapy.core.engine] INFO: Closing spider (finished)
2020-12-19 08:19:40 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 871,
 'downloader/request_count': 2,
 'downloader/request_method_count/GET': 2,
 'downloader/response_bytes': 1047,
 'downloader/response_count': 2,
 'downloader/response_status_count/200': 2,
 'elapsed_time_seconds': 41.143926,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2020, 12, 19, 11, 19, 40, 45332),
 'log_count/DEBUG': 5,
 'log_count/ERROR': 1,
 'log_count/INFO': 27,
 'request_depth_max': 1,
 'response_received_count': 2,
 'scheduler/dequeued': 2,
 'scheduler/dequeued/memory': 2,
 'scheduler/enqueued': 2,
 'scheduler/enqueued/memory': 2,
 'spider_exceptions/UnicodeDecodeError': 1,
 'start_time': datetime.datetime(2020, 12, 19, 11, 18, 58, 901406)}
2020-12-19 08:19:40 [scrapy.core.engine] INFO: Spider closed (finished)
2020-12-19 08:19:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET http